In [1]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import yaml

In [2]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [3]:
url = "bolt://localhost:7687"
driver = GraphDatabase.driver(url, auth=(PARAM["neo4j_username"], PARAM["neo4j_password"]))
gds = GraphDataScience(url, auth=(PARAM["neo4j_username"], PARAM["neo4j_password"]))

In [4]:
#### avoid-free supportive plants for potatoes by counting AVOIDS 

with driver.session() as session:
    result = session.run('MATCH (p1:Plant)-[r:HELPS]->(p:Plant {name: "potato"}) RETURN p1')
    supportive = [r["p1"]["name"] for r in result]
    print ("The supportive plants are: ", sorted(supportive))
    
    plant_antagonist_count = {"dummy": 0}

    while len(plant_antagonist_count) > 0:

        plant_antagonist_count = {}

        for plant_name in supportive:

            result = session.run(f'MATCH (p1:Plant {{name: "{plant_name}"}})-[r:AVOIDS]-(p:Plant) RETURN p')
            
            antagonists = set([r["p"]["name"] for r in result]).intersection(supportive)

            if len(antagonists) > 0:
                plant_antagonist_count[plant_name] = [len(antagonists), antagonists]
            
        
        print (plant_antagonist_count)

        if len(plant_antagonist_count) > 0:
            most_avoided = sorted(plant_antagonist_count.items(), key=lambda x: x[1][0], reverse=True)[0][0]
            print (most_avoided)
            supportive.remove(most_avoided)

    print ("After removing all the AVOIDS: ", sorted(supportive))



The supportive plants are:  ['alfalfa', 'basil', 'brassica', 'bush bean', 'cabbage', 'carrot', 'chive', 'clover', 'corn', 'crownvetch', 'dead nettle', 'fava bean', 'flax', 'garlic', 'horseradish', 'leek', 'lentil', 'lovage', 'lupin', 'marigold', 'mint', 'onion', 'oregano', 'pea', 'peanut', 'pole bean', 'soybean', 'tarragon', 'thyme', 'vetch', 'yarrow']
{'peanut': [4, {'onion', 'garlic', 'leek', 'chive'}], 'onion': [12, {'fava bean', 'soybean', 'peanut', 'alfalfa', 'lupin', 'bush bean', 'pole bean', 'lentil', 'crownvetch', 'clover', 'pea', 'vetch'}], 'alfalfa': [6, {'fava bean', 'garlic', 'leek', 'chive', 'onion', 'bush bean'}], 'brassica': [1, {'pole bean'}], 'fava bean': [5, {'garlic', 'leek', 'chive', 'onion', 'alfalfa'}], 'pea': [4, {'onion', 'garlic', 'leek', 'chive'}], 'thyme': [2, {'basil', 'tarragon'}], 'vetch': [4, {'onion', 'garlic', 'leek', 'chive'}], 'soybean': [5, {'garlic', 'leek', 'chive', 'onion', 'bush bean'}], 'cabbage': [1, {'garlic'}], 'lupin': [4, {'onion', 'garlic'

In [5]:
#### avoid-free supportive plants for potatoes with Degree Centrality from GDS

with driver.session() as session:
    result = session.run('MATCH (p1:Plant)-[r:HELPS]->(p:Plant {name: "potato"}) RETURN p1')
    supportive = [r["p1"]["name"] for r in result]
    print ("The supportive plants are: ", sorted(supportive))

    go = True

    while go == True:
        if gds.graph.exists("supportive_plants")["exists"]:
            gds.graph.drop(gds.graph.get("supportive_plants"))

        G, res = gds.graph.project.cypher('supportive_plants',
        f'MATCH (p1:Plant)-[r:HELPS]->(p:Plant {{name: "potato"}}) WHERE p1.name in {supportive} RETURN id(p1) AS id',
        f'MATCH (p1:Plant)-[r:AVOIDS]->(p2:Plant) WHERE p1.name in {supportive} AND p2.name in {supportive} RETURN id(p1) AS source, id(p2) AS target'
        )
        
        df = gds.degree.stream(G)
        l = gds.util.asNodes((df['nodeId'].to_list()))

        df['name'] = [x["name"] for x in l]

        most_avoided = df.loc[df["score"].idxmax()]
        most_avoided_name = df.loc[df["score"].idxmax()]["name"]
        most_avoided_score = df.loc[df["score"].idxmax()]["score"]

        supportive.remove(most_avoided_name)

        if df["score"].nlargest(2).iloc[1] == 0:
            go = False

    print ("After removing all the AVOIDS: ", sorted(supportive))


The supportive plants are:  ['alfalfa', 'basil', 'brassica', 'bush bean', 'cabbage', 'carrot', 'chive', 'clover', 'corn', 'crownvetch', 'dead nettle', 'fava bean', 'flax', 'garlic', 'horseradish', 'leek', 'lentil', 'lovage', 'lupin', 'marigold', 'mint', 'onion', 'oregano', 'pea', 'peanut', 'pole bean', 'soybean', 'tarragon', 'thyme', 'vetch', 'yarrow']
After removing all the AVOIDS:  ['cabbage', 'carrot', 'clover', 'corn', 'crownvetch', 'dead nettle', 'fava bean', 'flax', 'horseradish', 'lentil', 'lovage', 'lupin', 'marigold', 'mint', 'pea', 'peanut', 'pole bean', 'soybean', 'thyme', 'vetch', 'yarrow']
